In [15]:
import pandas as pd
import numpy as np

In [16]:
data=pd.read_csv('Glass.csv')

In [17]:
data.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.0,0.0,1
1,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.0,1
2,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.0,1
3,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.0,1
4,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.0,1


In [18]:
# chcekd for dataset shape
data.shape

(214, 10)

In [19]:
data.describe()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
count,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000,214.000000
mean,1.518365,13.407850,2.684533,1.444907,72.650935,0.497056,8.956963,0.175047,0.057009,2.780374
std,0.003037,0.816604,1.442408,0.499270,0.774546,0.652192,1.423153,0.497219,0.097439,2.103739
min,1.511150,10.730000,0.000000,0.290000,69.810000,0.000000,5.430000,0.000000,0.000000,1.000000
25%,1.516522,12.907500,2.115000,1.190000,72.280000,0.122500,8.240000,0.000000,0.000000,1.000000
50%,1.517680,13.300000,3.480000,1.360000,72.790000,0.555000,8.600000,0.000000,0.000000,2.000000
75%,1.519157,13.825000,3.600000,1.630000,73.087500,0.610000,9.172500,0.000000,0.100000,3.000000
max,1.533930,17.380000,4.490000,3.500000,75.410000,6.210000,16.190000,3.150000,0.510000,7.000000


In [20]:
data.duplicated().sum()

1

In [21]:
data.drop_duplicates(inplace=True)

In [22]:
data.reset_index(inplace=True)

In [23]:
# checked datatypes and null values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   213 non-null    int64  
 1   RI      213 non-null    float64
 2   Na      213 non-null    float64
 3   Mg      213 non-null    float64
 4   Al      213 non-null    float64
 5   Si      213 non-null    float64
 6   K       213 non-null    float64
 7   Ca      213 non-null    float64
 8   Ba      213 non-null    float64
 9   Fe      213 non-null    float64
 10  Type    213 non-null    int64  
dtypes: float64(9), int64(2)
memory usage: 18.4 KB


In [24]:
# chceked target values and its counts
data['Type'].value_counts()

2    76
1    69
7    29
3    17
5    13
6     9
Name: Type, dtype: int64

In [25]:
# now we split into features and target
x=data.iloc[:,:9]
y=data.iloc[:,[-1]]

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [27]:
scaler=StandardScaler()
sm=SMOTE()

In [28]:
# now split into training and testing set
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.30,random_state=42)
x_train=scaler.fit_transform(x_train)
x_test=scaler.fit_transform(x_test)
x_train,y_train=sm.fit_resample(x_train,y_train)

In [35]:
y_train.value_counts()

Type
1       52
2       52
3       52
5       52
6       52
7       52
dtype: int64

In [40]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold

In [41]:
# now we use knn and grid search cv for best params 
knn=KNeighborsClassifier()
kf=KFold(n_splits=10,shuffle=True,random_state=0)
params={'n_neighbors':[1,2,3,4,5,6,7,8]}
g_v=GridSearchCV(knn,param_grid=params,cv=kf,n_jobs=-1)
model=g_v.fit(x_train,y_train)

C:\Users\dell\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [42]:
model.best_params_

{'n_neighbors': 1}

In [43]:
model.best_score_

0.955141129032258

In [44]:
model.score(x_train,y_train)

1.0

In [45]:
model.score(x_test,y_test)

0.84375

In [36]:
y_pred=model.predict(x_test)

In [39]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           1       1.00      0.78      0.88        23
           2       0.75      0.86      0.80        21
           3       0.75      0.75      0.75         4
           5       1.00      0.80      0.89         5
           6       0.67      1.00      0.80         2
           7       0.82      1.00      0.90         9

    accuracy                           0.84        64
   macro avg       0.83      0.86      0.84        64
weighted avg       0.87      0.84      0.85        64

